# HuggingFace_PipeLine

https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct-GGUF

qwen2.5-0.5b-instruct-q4_k_m.gguf

Qwen2.5 is the latest series of Qwen large language models. For Qwen2.5, we release a number of base language models and instruction-tuned language models ranging from 0.5 to 72 billion parameters. Qwen2.5 brings the following improvements upon Qw
Significant improvements in instruction following, generating long texts (over 8K tokens), understanding structured data (e.g, tables), and generating structured outputs especially JSON. More resilient to the diversity of system prompts, enhancing role-play implementation and condition-setting for chatbots.
Long-context Support up to 128K tokens and can generate up to 8K tokens.
Multilingual support for over 29 languages, including Chinese, English, French, Spanish, Portuguese, German, Italian, Russian, Japanese, Korean, Vietnamese, Thai, Arabic, and more.

### 언어모델

In [ ]:
!pip uninstall -y llama-cpp-python llama-cpp-python-cu121 llama-cpp-python-cu118
!pip install llama-cpp-python
# GTX 을 위하여... ㅠㅠ

Found existing installation: llama_cpp_python 0.3.16
Uninstalling llama_cpp_python-0.3.16:
  Successfully uninstalled llama_cpp_python-0.3.16


     ---------------------------------------- 0.0/50.7 MB ? eta -:--:--
     - -------------------------------------- 1.6/50.7 MB 16.6 MB/s eta 0:00:03
     ---- ----------------------------------- 5.2/50.7 MB 17.7 MB/s eta 0:00:03
     ------- ------------------------------- 10.0/50.7 MB 19.4 MB/s eta 0:00:03
     ---------- ---------------------------- 13.6/50.7 MB 19.0 MB/s eta 0:00:02
     ------------ -------------------------- 15.7/50.7 MB 17.1 MB/s eta 0:00:03
     ------------- ------------------------- 18.1/50.7 MB 16.3 MB/s eta 0:00:03
     -------------- ------------------------ 18.9/50.7 MB 14.7 MB/s eta 0:00:03
     --------------- ----------------------- 20.2/50.7 MB 12.7 MB/s eta 0:00:03
     ---------------- ---------------------- 21.0/50.7 MB 11.7 MB/s eta 0:00:03
     ----------------- --------------------- 22.8/50.7 MB 11.4 MB/s eta 0:00:03
     ------------------- ------------------- 25.7/50.7 MB 11.7 MB/s eta 0:00:03
     ---------------------- ---------------- 28.

  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.11.6 using CMake 4.1.0 (wheel)
      *** Configuring CMake...
      2025-08-28 15:30:15,114 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\venus\AppData\Local\Temp\tmpf3an4j0b\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error or

In [18]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="Qwen/Qwen2.5-0.5B-Instruct-GGUF", filename = "qwen2.5-0.5b-instruct-q4_k_m.gguf") # 다운로드 파일의 local path 저장
print(model_path)

C:\Users\venus\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B-Instruct-GGUF\snapshots\9217f5db79a29953eb74d5343926648285ec7e67\qwen2.5-0.5b-instruct-q4_k_m.gguf


In [24]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
import os

# 모델 다운로드 & 로드
model_path = hf_hub_download(
    repo_id="Qwen/Qwen2.5-0.5B-Instruct-GGUF",
    filename="qwen2.5-0.5b-instruct-q4_k_m.gguf"
)

llm = Llama(
    model_path=model_path,
    n_ctx=2048,     
    n_threads= max(4, os.cpu_count() - 2),
    n_batch=256,    
    n_gpu_layers=0,  # CPU 전용 GTX 1050...
    verbose=False      # ← 로그 숨김
)

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [29]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
import os, json
from pathlib import Path

SAVE_PATH = Path("story.json")

def load_story():
    if SAVE_PATH.exists():
        with open(SAVE_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_txt(story, txt_path="story.txt"):
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write("소설 초안\n\n")
        for i, p in enumerate(story, 1):
            f.write(f"[{i}] {p}\n\n")

def save_story(story):
    with open(SAVE_PATH, "w", encoding="utf-8") as f:
        json.dump(story, f, ensure_ascii=False, indent=2)

def continue_paragraph(paragraphs, genre="판타지 미스터리", style="감성 묘사"):
    guide = (
        f"[역할] 한국어 문학 소설가\n"
        f"[장르] {genre}\n"
        f"[스타일] {style} / 문장은 자연스럽고 연결성 있게, 과장된 추상 표현 금지\n"
        f"[제약] 한 문단만, 줄바꿈 0~1회, 반복 표현 자제\n"
    )
    sofar = "\n\n".join(paragraphs)
    prompt = (
        f"{guide}\n"
        f"지금까지의 이야기:\n{sofar}\n\n"
        f"[다음 문단]\n"
    )
    out = llm(
        prompt,
        max_tokens=140,
        temperature=0.6,
        top_p=0.8,
        top_k=40,
        repeat_penalty=1.07,
        stop=["\n\n\n","###","</s>"]
    )
    text = out["choices"][0]["text"].strip()
    return text.split("\n\n")[0].strip()

story = load_story()
if not story:
    start = input("소설 첫 문장을 입력하세요: ")
    story = [start]

turns = int(input("이번에 몇 문단을 더 이어 쓸까요? (예: 2): ") or "2")
for _ in range(turns):
    story.append(continue_paragraph(story))

print("\n====== 📖 현재까지의 이야기 (통합 출력) ======\n")
for i, para in enumerate(story, 1):
    print(f"[{i}] {para}\n")

# 저장
save_story(story)
print(f"\n✅ 저장 완료: {SAVE_PATH.resolve()}")

save_txt(story)
print("✅ story.txt 저장 완료")
# llm(prompt, ... )


====== 📖 현재까지의 이야기 (통합 출력) ======

[1] 샛별이라는 고양이는 마법사가 키우는 고양이였다. 어느날 마법사가 말했다. 샛별아 잠시동안 나는 집을 떠나게 되었단다. 그동안 집을 지켜줄 수 있겠니? 샛별이는 아무것도 모른채 야옹이라고 대답했다. 그때까지 모르고 있었다. 샛별이의 주인이 돌아오지 않을 것이란 사실을... 샛별이는 하루가 지나고 이틀이 지나고 언젠간 주인이 돌아올 것이라 믿으며 꼬박 1년을 기다렸다. 그러나 주인은 돌아오지 않았다. 주인이 자신을 버렸으리라 생각하지 않는 순수한 고양이인 샛별은 주인이 길을 잃어버렸으리라 생각했다. 그래서 샛별이는 집을 나섰다. 주인을 찾기위해, 주인을 찾아서 집으로 함께 돌아오기 위해 말이다. 샛별이가 집을 나서고 처음으로 방문한 곳은 언덕아래 마을이었다. 언덕아래 마을은 사람들이 무리지어 살고 있었는데 그곳에 사는 사람들은 고양이를 싫어하기 때문에 주인이 가지말라고 당부했었다. 그러나 샛별이는 주인을 찾기 위해 사람이 사는 마을로 가야만 한다고 생각했다. 마을에 도착하자 사람들은 샛별이를 바라보며 기분나쁘다는 표정을 지었다. 순수한 샛별이는 괜찮으리라 생각하며 걸어나갔다. 그때 마을의 한 사람이 멀리서 샛별이에게 돌을 던졌고, 아무것도 모르는 샛별이가 돌을 맞으려는 순간...! 옆에서 누군가 돌을 막아주었다. 샛별이가 놀란 눈으로 올려다보자 그곳엔 빨간머리를 가진 한 소녀가 있었다.

[2] [제약] 한 문단만, 줄바꿈 0~1회, 반복 표현 자제

[3] 나는 그 소녀에게 말했다. 당신이 고양이를 싫어하는 이유가 무엇일까? 당신은 고양이는 심지어 가족과도 사랑하지 않는 분들이 있다는 것을 알고 있는 거다. 하지만 이들은 고양이를 사랑하는 사람들과 같은 사람들을 위해 싸우는 거야. 고양이의 손을 잡는 것은 그저 고양이가 싫어하는 사람들에게만 가능하다는 거야. 고양이는 심지어 가족과도 사랑하지 않는 사람들을 위해 싸우는 거야. 그들은 고양이를 사랑하는 사람들과 같은 사람들을 위해


✅ 저장 